## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
import os
# 本範例不需使用 GPU, 將 GPU 設定為 "無"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model=keras.applications.Xception(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)

'''Resnet 50 架構'''
#model=keras.applications.ResNet50(include_top=False, weights='imagenet',
#                                    input_tensor=input_tensor,
#                                    pooling=None, classes=10)
model.summary()


Using TensorFlow backend.


Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

## 添加層數

In [3]:
x = model.output

'''可以參考Cifar10實作章節'''
x = GlobalAveragePooling2D()(x)
predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 134


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, activation="softmax")`
  """


## 鎖定特定幾層不要更新權重

In [4]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 501s 10ms/step - loss: 1.4473 - accuracy: 0.4822
Epoch 2/100
50000/50000 [==============================] - 501s 10ms/step - loss: 0.9903 - accuracy: 0.6562
Epoch 3/100
50000/50000 [==============================] - 501s 10ms/step - loss: 0.8767 - accuracy: 0.6951
Epoch 4/100
50000/50000 [==============================] - 501s 10ms/step - loss: 0.7799 - accuracy: 0.7288
Epoch 5/100
50000/50000 [==============================] - 501s 10ms/step - loss: 0.7000 - accuracy: 0.7541
Epoch 6/100
50000/50000 [==============================] - 501s 10ms/step - loss: 0.6278 - accuracy: 0.7810
Epoch 7/100
50000/50000 [==============================] - 501s 10ms/step - loss: 0.5582 - accuracy: 0.8064
Epoch 8/100
50000/50000 [==============================] - 501s 10ms/step - loss: 0.4983 - accuracy: 0.8265
Epoch 9/100
50000/50000 [==============================] - 501s 10ms/step - loss: 0.4397 - accuracy: 0.8443
Epoch 10/100
50000/50000 [==

50000/50000 [==============================] - 499s 10ms/step - loss: 0.0643 - accuracy: 0.9778
Epoch 77/100
50000/50000 [==============================] - 498s 10ms/step - loss: 0.0633 - accuracy: 0.9789
Epoch 78/100
50000/50000 [==============================] - 499s 10ms/step - loss: 0.0621 - accuracy: 0.9794
Epoch 79/100
50000/50000 [==============================] - 499s 10ms/step - loss: 0.0624 - accuracy: 0.9788
Epoch 80/100
50000/50000 [==============================] - 499s 10ms/step - loss: 0.0618 - accuracy: 0.9789
Epoch 81/100
50000/50000 [==============================] - 499s 10ms/step - loss: 0.0595 - accuracy: 0.9800
Epoch 82/100
50000/50000 [==============================] - 498s 10ms/step - loss: 0.0613 - accuracy: 0.9796
Epoch 83/100
50000/50000 [==============================] - 499s 10ms/step - loss: 0.0565 - accuracy: 0.9803
Epoch 84/100
50000/50000 [==============================] - 498s 10ms/step - loss: 0.0597 - accuracy: 0.9807
Epoch 85/100
50000/50000 [======